### Working with Financial Data

Publicly traded companies are classified into one of 11 industry sectors, called GICS (Global Industry Classification Standard). Investors use these classes to design and construct portfolios, compare companies, evaluate industries, and isolate regional from global trends.

There are 11 GICS sectors

* Communication services
* Consumer discretionary
* Consumer staples
* Energy
* Financials
* Health care
* Industrials
* Information technology
* Materials
* Real estate
* Utilities


In this notebook we'll work with financial data, grouped by sector, to analyze different trends in companies and sectors listed on the S&P 500. 


Begin by loading the GICS sector mappings. They've been provided to you in a file called `gics.csv`


In [36]:
import pandas as pd
path = '/Users/dgriffis/code/glg/python-data-analysis-crash-course/data/'
gicsdf = pd.read_csv(path+'gics.csv')
gicsdf['Name'] = gicsdf['Name'].str.lower() # doing this for a later key match
#gicsdf.head()
print(gicsdf)

    Code                    Name
0     50  communication services
1     25  consumer discretionary
2     30        consumer staples
3     10                  energy
4     40              financials
5     35             health care
6     20             industrials
7     45  information technology
8     15               materials
9     60             real estate
10    55               utilities


Load the S&P 500 sector mappings and take a look at the data. The file is called `sp500-sector.csv`

In [37]:
sp500df = pd.read_csv(path+'sp500-sector.csv')
sp500df['Sector'] = sp500df['Sector'].str.lower() #doing this for a later key match
sp500df.head()

,Symbol,Name,Sector
0,MMM,3M Company,industrials
1,AOS,A.O. Smith Corp,industrials
2,ABT,Abbott Laboratories,health care
3,ABBV,AbbVie Inc.,health care
4,ACN,Accenture plc,information technology


Create a dataframe that has the S&P 500 stock symbols with the appropriate sector number. For example, `3M`, symbol `MMM` is in `industrials` which corresponds to sector 20.

In [38]:
sp500Gics = pd.merge(gicsdf,sp500df, left_on='Name', right_on='Sector', how='right')
sp500Gics = sp500Gics[['Name_y','Symbol','Sector','Code']]
sp500Gics.columns=['Name','Symbol','Sector','Code']
sp500Gics['Sector'] = sp500Gics['Sector'].str.title()
#sp500Gics['Code'][sp500Gics['Sector'] == 'Telecommunication Services'] = 50 #This causes a SettingwithCopyWarning
sp500Gics.loc[sp500Gics['Sector'] == 'Telecommunication Services','Code'] = 50 
sp500Gics.head()

sp500Gics[sp500Gics.Symbol == 'MMM']

,Name,Symbol,Sector,Code
277,3M Company,MMM,Industrials,20.0


Working with strings in pandas can be much slower than working with integers and floats. Securities are typically represented by a security id, a numerical representation of the security. Ids have been provided in the file `sp500-ids.csv`. Load these into a dataframe and then generate a new dataframe that contains the columns `security_id` and `sector`.

In [39]:
sp500Ids = pd.read_csv(path+'sp500-ids.csv')
sp500Ids.head()


,Symbol,Security_id
0,MMM,1
1,AOS,2
2,ABT,3
3,ABBV,4
4,ACN,5


In [40]:
sp500SectorTickerId = pd.merge(sp500Gics, sp500Ids, on='Symbol')
sp500SectorTickerId = sp500SectorTickerId[['Sector','Security_id']]
sp500SectorTickerId.head()

,Sector,Security_id
0,Consumer Discretionary,9
1,Consumer Discretionary,31
2,Consumer Discretionary,54
3,Consumer Discretionary,62
4,Consumer Discretionary,72


Load the file `sp500-5yr.csv` into a dataframe. This file contains 5 years worth of historical prices for securities listed on the S&P 500. Convert the symbols into security ids, and bring in the sector colums as well.

In [41]:
sp5005yr = pd.read_csv(path+'sp500-5yr.csv')
#sp5005yr
#merge the merge
sp5005yrMerged = pd.merge(pd.merge(sp5005yr, sp500Ids, left_on='symbol', right_on='Symbol'),sp500SectorTickerId, on='Security_id')
sp5005yrMerged = sp5005yrMerged.drop(columns=['symbol','Symbol'])
sp5005yrMerged.head()


,date,open,high,low,close,volume,Security_id,Sector
0,2013-02-08,15.07,15.12,14.63,14.75,8407500,33,Industrials
1,2013-02-11,14.89,15.01,14.26,14.46,8882000,33,Industrials
2,2013-02-12,14.45,14.51,14.10,14.27,8126000,33,Industrials
3,2013-02-13,14.30,14.94,14.25,14.66,10259500,33,Industrials
4,2013-02-14,14.94,14.96,13.16,13.99,31879900,33,Industrials


Determine which sectors had the best performance for the past year, past three years and past five years. Note the start and end dates on the dataset.

In [42]:
#close at start of year compared to close at end of year
#for each security get the start and end close
df1_yr = sp5005yrMerged.loc[sp5005yrMerged['date'].isin(['2017-02-07', '2018-02-07'])]
df1_yr = df1_yr.reset_index(drop=True)
df1_yr


,date,open,high,low,close,volume,Security_id,Sector
0,2017-02-07,45.750,46.1300,45.0100,45.17,6119960,33,Industrials
1,2018-02-07,50.910,51.9800,50.8900,51.40,4845831,33,Industrials
2,2017-02-07,130.540,132.0900,130.4500,131.53,38183841,52,Information Technology
3,2018-02-07,163.085,163.4000,159.0685,159.54,51608580,52,Information Technology
4,2017-02-07,161.210,162.0600,157.7800,159.18,1191286,9,Consumer Discretionary
5,2018-02-07,112.090,114.5799,109.9000,109.93,2163545,9,Consumer Discretionary
6,2017-02-07,60.650,60.8700,60.4000,60.56,4210335,4,Health Care
7,2018-02-07,111.520,116.8500,111.3800,113.62,10030837,4,Health Care
8,2017-02-07,89.850,91.6400,89.7100,90.33,2108481,40,Health Care
9,2018-02-07,91.600,95.3400,91.1000,94.22,2509484,40,Health Care


In [43]:
df3_yr = sp5005yrMerged.loc[sp5005yrMerged['date'].isin(['2015-02-06', '2018-02-07'])]
df3_yr = df3_yr.reset_index(drop=True)
df3_yr

df5_yr = sp5005yrMerged
df5_yr.head()

,date,open,high,low,close,volume,Security_id,Sector
0,2013-02-08,15.07,15.12,14.63,14.75,8407500,33,Industrials
1,2013-02-11,14.89,15.01,14.26,14.46,8882000,33,Industrials
2,2013-02-12,14.45,14.51,14.10,14.27,8126000,33,Industrials
3,2013-02-13,14.30,14.94,14.25,14.66,10259500,33,Industrials
4,2013-02-14,14.94,14.96,13.16,13.99,31879900,33,Industrials


In [44]:

#combine the two rows for each security into one row 
df1_yr['Performance'] = df1_yr['close'].groupby(df1_yr['Security_id']).pct_change() * 100
df1_yr = df1_yr[df1_yr['Performance'].notnull()]
df1_yr


,date,open,high,low,close,volume,Security_id,Sector,Performance
1,2018-02-07,50.910,51.9800,50.8900,51.40,4845831,33,Industrials,13.792340
3,2018-02-07,163.085,163.4000,159.0685,159.54,51608580,52,Information Technology,21.295522
5,2018-02-07,112.090,114.5799,109.9000,109.93,2163545,9,Consumer Discretionary,-30.939817
7,2018-02-07,111.520,116.8500,111.3800,113.62,10030837,4,Health Care,87.615588
9,2018-02-07,91.600,95.3400,91.1000,94.22,2509484,40,Health Care,4.306432
11,2018-02-07,58.600,59.8500,58.5000,58.67,6872663,3,Health Care,36.791793
13,2018-02-07,154.220,158.9300,153.0700,155.15,2918659,5,Information Technology,33.842305
15,2018-02-07,193.870,196.2400,192.0200,192.34,2988024,8,Information Technology,67.310369
17,2018-02-07,86.750,87.9500,85.2900,85.35,4555784,45,Information Technology,11.846416
19,2018-02-07,42.310,43.0500,42.1700,42.19,4587806,55,Consumer Staples,-5.382373


In [45]:
#df1_yr.dtypes
df1_yr.groupby(['Sector'])['Performance'].median().sort_values(ascending=False)

Sector
Information Technology        29.940394
Health Care                   20.188876
Financials                    18.400520
Industrials                   18.034414
Consumer Discretionary        16.383192
Materials                     14.051369
Utilities                      0.926128
Energy                        -2.194999
Consumer Staples              -2.669081
Real Estate                   -9.694978
Telecommunication Services   -10.189689
Name: Performance, dtype: float64

Imagine two portfolios, one comprised of the securites represented by even ids and one by odd ids. Which portfolio has had the best performance over the 1,3 and 5 year time windows?


Combine these two portfolios into a single portfolio. What was the overall performance of the SP500 for the same time periods?
